# Case Study 4
## Description 
As a user of satellite data, I want to compare high-quality ground-based data from multiple sites with a satellite product, so that I can assess its precision and accuracy for estimating the same variables at other sites.  
## Case Breakdown 
- **Actors:** Satellite Data User
- **Goals:** Finding correlations between satellite data and ground-based data
- **Scope:** National, point-based
- **Language:** Python
- **Extra:** TBD
## Generalised case
I want to compare measurements from a satellite data product with actual on-ground measurements from different sites and report mean and standard deviation for the error.
## Comparable cases
- I want to compare local weather station values at APPN and/or TERN sites with the associated daily measurememts from national weather datasets (BOM).
## Stakeholders 
- **Name:** TBD
- **Contact:** donald.hobern@adelaide.edu.au


## Data Sources - <span style="color:red">DRAFT</span>
**UPDATED PLAN 2025-02-07**
This case study is conceptually simple. We just need a spatial data product (not necessarily satellite imagery) that provides estimates of some variable for particular times and places and some local points of truth that provide measurements of local values for the same variable.

The obvious examples are weather data. We could calculate local values at one or more sites for one or more dates that should match the values in the spatial data product. We could then assess the mean error and the standard deviation of the errors for those times and places. These errors could be calibrated as fractions of the total range across which the variable is measured.

The problem is that I'm not sure any of our current focus sites are sharing weather data. Need to check with Rakesh.


<span style="color:silver">
**Reference:** https://appf-central.atlassian.net/l/cp/7yrLwqX1
Data Source 1: SILO Climate DATABASE 
- **Description:** SILO Climate Database for environmental data.
- **Format:** TBD
- **Access Method:** https://www.longpaddock.qld.gov.au/silo/
Data Source 2: Extreme Weather Data (BOM)
- **Description:** Extreme weather data from the Bureau of Meteorology (BOM).
- **Format:** TBD
- **Access Method:** http://www.bom.gov.au/akamai/https-redirect.html
Data Source 1: On-Ground Site
- **Description:** Data from where the chickpeas come from.
- **Format:** TBD
- **Access Method:** TBD
Data Catalog - DRAFT
Describe what kind of data will be using 
</span>

## Data Transformation - <span style="color:red">DRAFT</span> 
Describe the steps take to process the data. 

In [4]:
## CODE EXAMPLE 
## THIS
## is 
## the part of 
## CODE

## DataCube Generation - <span style="color:red">DRAFT</span> 
Describe the steps take to process the data. 

In [5]:
## CODE EXAMPLE 
## THIS
## is 
## the part of 
## CODE

## Data Analysis/Visulisation - <span style="color:red">DRAFT</span> 
Describe the steps take to process the data. 
### Requirments 
- Provide the aggregate spatial data into excel files
- Pictures should be presented by URL

In [6]:
## CODE EXAMPLE 
## THIS
## is 
## the part of 
## CODE

## NonFunctional Requirements - <span style="color:red">DRAFT</span> 
### Welcome PAGE
#### Objectives
- Showing users what they can do and get and how they can proceed.
- Showing the most updeted information. 
#### Requirements 
- Create a simple UI/Template for user input their requirements
  - showing the normal process of querying data and provide sample data
  - showing the sample output users are expected to have
  - showing the resources users can use
- Creating a 'HELP' Page for users to looking for what kind of attributes they can use
  - Including all the attributes they can query from the data sources
  - Including all the functions users can use and expecting outputs of those query
  - Including informations for asking questions and reaching for help.
- Creating a 'Annoucement' Page for showing the most updated information of the project
  - Including versions/Time/Detail

### User Query 
#### Objectives
- take input from users and process with related data
- Analyze the data and give feedback of what is expected
  - Error Control
  - Time estimation
  - Output Estimatation
#### Requirements 
- Using basic Query language to query DATA
    > Select Phenotype*
    > from On-GroundSite
    > Where CropType = 'Chickpeas'

    > Select Environmental
    > from SILO
    > where weather ='Extreme'- ERROR Control
- Estimation for time & Output
    > ----> Estimated Files intotal: 500 file 5GB
    > ----> Estimated Time:5 mins 50%
    > ----> Noted it may depends on your download speed balabla.
### Error Control
#### Objectives
- State clear what is happening while using try to query
- Give advice & suggestions

### Extras
#### Non-Functional Requirements
- Ensure querying time is within a set time (less than 1 second).
- Create a waiting process for querying if time is not stable.
- Present outputs in a consistent and organized manner.
- Handle unexpected errors gracefully, showing error codes and continuing with other feasible data.